# Disclaimer

These are tools developed to help me determine when my intellectual property is posted without my permission online. 
These are tools created to help me ensure that content uploaded by students to the site do not infringe my intellectual property rights.

They simply automate the process of searching the site and thus make it easier to determine which items infringe rights and which do not.

They are extremely rough and probably not very user friendly. That said, I am making them public in case others find them useful.  

These tools are provided in the hope that they will be useful to those seeking to protect their intellectual property, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

The tools are provided here with no warranty or claim that their use is consistent with any law, policy, terms of use policy, or any other constraint. IF YOU CHOOSE TO USE THEM, YOU AGREE TO REVIEW AND COMPLY WITH ALL RELEVANT POLICIES, INCLUDING SITE TERMS OF SERVICE, AND YOU AGREE THAT YOUR USE IS SOLELY AT YOUR OWN RISK.

Questions, requests, complaints, or other feedback (or pull-requests!) are welcome through this project's github page
        
        https://github.com/HatakoHaterson/CourseZero

Please understand that this is not an active project for me; I will monitor the project on github very infrequently.

-- 復讐者


View a static image of the main notebook (does not run) on Jupyter: 
    
    https://nbviewer.jupyter.org/github/HatakoHaterson/CourseZero/blob/master/Notebooks/Gather%20information.ipynb?flush_cache=true

Or run it on your own mini-server using Binder

    https://mybinder.org/v2/gh/HatakoHaterson/CourseZero/master?filepath=Notebooks%2FGather%20information.ipynb


# Get started

Once the notebook has opened in binder, a toolbar will appear at the top of the notebook containing several buttons. The 'run' button is the most important.

After you've opened the notebook in binder, click the run button above a few times. Each click runs the code in the highlighted cell. You can also press shift+return to run it. You can change which cell is highlighted with the up and down arrows or by just clicking on it.

Try it with the next cell:

In [ ]:
print("You've successfully run this cell!")

Since these are rough-and-ready tools, I've left in much of the code. If you want to follow along, I've left comments to myself which identify the various steps. These are either single lines which start with a '#', like this: 
    
    # Query the server for records
    code code code
    
Or, multiple lines enclosed with three double quotes: 
    
    """blah blah
    tacos are good
    """  

You'll be able to tell that a cell is running by an asterisk appearing in the brackets to the left ( [ * ]). Some tasks may take a couple of minutes to run, so it may appear frozen.

I haven't set it up with an easy way to reset things. If you want to search a different campus or different set of departments, use the 'restart kernel' button above (to the right of the run button).

The next cell will grab and import all the code that operates behind the scenes. (If you're curious, you can check it out here: https://github.com/HatakoHaterson/CourseZero)

In [3]:
# for binder
# !pip install CourseZero

# for local testing
%cd ../

import json
import pandas as pd

import requests
from bs4 import BeautifulSoup

import environment as env
from CourseZero.FileSystemTools import getSystemRoot

from IPython.display import display
from ipywidgets import widgets

from CourseZero.RequestTools import get_docs_for_campuses, get_docs_for_campus
from CourseZero.Store import DataStore
from CourseZero.Widgets.CampusAndProfIdentifiers import make_campus_selector
from CourseZero.FileSystemTools import getSystemRoot
from CourseZero.DataStorageTools import load_campus_id_data

/Users/adam/Dropbox


In [9]:
import sys

def t():
    this_function_name = sys._getframe().f_code.co_name
    print(this_function_name)
t()

t


In [24]:

def func_timer_decorator(func):
    def func_wrapper(*args, **kwargs):
        with EnterExitLog(func.__qualname__):
            return func(*args, **kwargs)

    return func_wrapper

In [44]:
def prop_inspector_dec(func):
    def func_wrapper(*args, **kwargs):
        cls_name = func.__qualname__.split('.')[0]
        cls_prop_name = "_{}".format(func.__name__)
        print(cls_name, cls_prop_name)
        if getattr(args[0], cls_prop_name) is None:
            print('oops')
        return func(*args, **kwargs)

    return func_wrapper


class T(object):
    _p = None
    
    def __init__(self):
        pass

    @property
    @prop_inspector_dec
    def p(cls):
        return cls._p


In [45]:
t = T()

# getattr(t, '_p')
t.p

T _p
oops


# Select your campus


Running the next cell will display a selection menu. Use it to select the CSU you want to search.

NB, for now this will only work with CSU campuses

In [5]:
# Load the campus ids from a file
# csu_id_data = '{}/csu_ids.json'.format(env.DATA_FOLDER)
campus_ids = load_campus_id_data(env.DEFAULT_CSU_ID_FILE)
csu_names = [c['name'] for c in campus_ids]
# Display the selector
make_campus_selector(DataStore, campus_ids)

Dropdown(description='Campus', layout=Layout(width='80%'), options=('California State University Bakersfield',…

# Search for courses on the selected campus

The next couple of cells will actually search for courses at the selected campus.

NB, when the search starts it will display a message like "Searching for California State University Dominguez hills". You may need to wait a few seconds for it to complete.

In [6]:
def parse_counts(row):
    """The server returns a field with a list of 
    the various kinds of documents that have been uploaded.
    This parses those counts out and adds them as columns to the frame
    """
    for c in row['doc_counts']:
        row[c['category']] = c['count']
    return row
    
def parse_json_into_df(json_data):
    """Parse json data into a usable dataframe
    For now: Control which columns are kept by commenting out in to_keep
    """
    to_keep = [
#         '@timestamp', 
     'canonical_course_id', 
#         'country', 
        'course_id',
       'course_info', 
        'course_name', 
        'course_num', 
#         'course_pk', 
        'dept_acro',
       'dept_id', 
        'dept_name', 
        'doc_counts', # if this isn't removed, can't dedupe
#         'location', 
        'prof_id',
       'prof_name', 
#         'school_aliases', 
#         'school_id', 
        'school_name',
#        'subdivision', 
        'term', 
        'title', 
        'total_doc_count', 
        'type', 
        'url',
       'verified', 
        'year'
    ]

    data = []

    for r in json_data:
        try:
            if r['total'] >0: 
                for row in r['results']:
                    data.append(row)

        except Exception as e:
            pass
    
    data = pd.DataFrame(data)
    # Filter out unneeded columns
    data = data[to_keep]
    # Add the counts of each kind of document to the frame as a column
    data = data.apply(lambda r: parse_counts(r), axis=1)
    # Drop the counts column so can de-dupe (list isn't hashable)
    data.drop(['doc_counts'], axis=1, inplace=True)
    data.drop_duplicates(inplace=True)
    return data

# Data frame operations
def get_departments(frame):
    """Extract list of departments from the results frame"""
    depts = list(set(frame.dept_acro.tolist()))
    depts.sort()
    return depts

def normalize_prof_name(prof_name):
    return prof_name.strip().upper()

# Frame filtration operations
def filter_by_dept_abbrevs(frame):
    """Uses the list of departments defined by the user to
    return those departments from the results frame"""
    return frame[frame['dept_acro'].isin(DataStore.departments)]

def get_by_course_id(frame, course_ids):
    """Finds and reeturns a course or list of courses from the results
    frame"""
    if type(course_ids) is not list:
        course_ids = list(course_ids)
    return frame[frame['course_id'].isin(course_ids)]


# Query for results and parse them into a dataframe
json_data = get_docs_for_campus(DataStore.campus_id, DataStore.campus_name)
data = parse_json_into_df(json_data)
print("{} courses identified ".format(len(data)))
depts = get_departments(data)

Searching for None


KeyError: "['canonical_course_id' 'course_id' 'course_info' 'course_name'\n 'course_num' 'dept_acro' 'dept_id' 'dept_name' 'doc_counts' 'prof_id'\n 'prof_name' 'school_name' 'term' 'title' 'total_doc_count' 'type' 'url'\n 'verified' 'year'] not in index"

# Select department names / abbreviations

The site does not seem to standardize the departments students enter. Thus your class may be identified in some rather odd ways.

Run the next cell and select all the possible departments using the buttons which display.

In [ ]:
def make_dept_b(dept):
    b = widgets.Button(description=dept, button_style='primary')
    def handle(event):
        if dept in DataStore.departments:
            DataStore.remove_department(dept)
            b.button_style='primary'
        else:
            DataStore.add_department(dept)
            b.button_style='success'
    b.on_click(handle)
    return b


def split_list(a_list):
    half = len(a_list)//2
    return a_list[:half], a_list[half:]

def dept_selection(depts):
    buttons = []
    for dept in depts:
        buttons.append(make_dept_b(dept))

    b1, b3 = split_list(buttons)
    b1, b2 = split_list(b1)
    b3, b4 = split_list(b3)
    
    display(widgets.HBox([widgets.VBox(b1), widgets.VBox(b2), widgets.VBox(b3), widgets.VBox(b4)]))

    
# Create the department selection buttons
dept_selection(depts)

# Display all courses with uploaded materials for the selected departments

Run the next cell and select any of the courses which look like they may be yours

In [ ]:
def make_selection_text(row):
    """Creates the text displayed in the course selection button"""
    t = "{prof_name} ---- {dept_acro} {course_num} ----  {course_name} ----  {course_info}"
    return t.format(**row.to_dict())

def make_course_b(row):
    """Creates a button for the course defined in the row. 
    Sets a handler on the button to toggle whether the course is selected
    in the data store
    """
    layout = widgets.Layout(width='90%')
    b = widgets.Button(description=make_selection_text(row), button_style='primary', layout=layout)
    
    def handle(event):
        if row['course_id'] in DataStore.course_ids:
            DataStore.remove_course(row['course_id'])
            b.button_style='primary'
        else:
            DataStore.add_course(row['course_id'])
            b.button_style='success'
    b.on_click(handle)
    return b


# Get the selected departments
filtered = filter_by_dept_abbrevs(data)
# Make the buttons
course_buttons =[]
for i, r in filtered.iterrows():
    course_buttons.append(make_course_b(r))

display(widgets.VBox(course_buttons))

# Display material uploaded for selected courses

The following cells will query the site and retrieve the url's for all files that students have uploaded for the courses you've selected.

You will need the file url's to request that the site remove the content via this website: https://www.coursehero.com/copyright-infringement/

Remember that the url's this tool returns are simply all the files uploaded for the selected courses. 

*You must review the url's to determine that they infringe your rights.* 

Students may have uploaded their own intellectual property related to your course. You will infringe their rights if you request that their intellectual property be removed.

In [ ]:
from CourseZero.RequestTools import get_content
CHBASE = "https://www.coursehero.com{}"


def get_file_links_from_course_page(course_page_url):
    """Retrieves the page that would be displayed if you went to the
    relevant page on the site, then filters out all the links to files
    since this is what you need to file the request
    """
    files = []

    html_content = get_content(CHBASE.format(course_page_url))
    soup = BeautifulSoup(html_content, 'html.parser')

    for j in soup.findAll('a'):
        try:
            dest = j['href']
            fnd = "/file/"
            if dest[: len(fnd)] == fnd:
                files.append(CHBASE.format(dest))
        except Exception as e:
            pass

    files = list(set(files))
    return files

def show_selected(frame):
    """Handles displaying the urls and information that has been retrieved"""
    selected = get_by_course_id(frame, DataStore.course_ids)
    for i, r in selected.iterrows():
        files = get_file_links_from_course_page(r['url'])
        t = "{prof_name} ---- {dept_acro} {course_num} ----  {course_name} ----  {course_info}"
        print(t.format(**r.to_dict()))
        for f in files:
            print(f)

show_selected(data)